# Amazon SNS (Simple Notification Service)

<img src="../_assets/aws_service_icons/sns.svg" width="80" alt="Amazon SNS">

## Goals
- Understand what **SNS** is
- Know what it’s practically used for
- See a simple **AWS SDK pseudo-code** example (not executed)


## What is SNS?

**Amazon SNS** is a fully managed **publish/subscribe (pub/sub)** messaging service.

- Producers (**publishers**) send messages to a **topic**.
- Consumers (**subscribers**) receive those messages through a delivery protocol.
- SNS handles **fan-out** (one message delivered to many subscribers) and retries.

Common subscription protocols/targets include: **SQS**, **Lambda**, **HTTP/S endpoints**, **email**, and **SMS**.


## What is SNS used for (practically)?

SNS is commonly used to **decouple systems** and to deliver **event notifications** to multiple downstream consumers.

Typical use cases:
- **Fan-out**: publish once → deliver to multiple SQS queues / Lambda functions / webhooks
- **Alerts & notifications**: CloudWatch alarms → SNS → email/SMS/on-call tooling
- **Workflow triggers**: events emitted by one service trigger background processing elsewhere
- **Broadcasting domain events**: e.g., `order_created`, `model_training_completed`

Operationally, SNS works best when subscribers are built to handle **at-least-once delivery** and **retries** (idempotency is important).


## Using SNS with the AWS SDK (pseudo-code)

Below is a minimal, **non-executable** sketch using an AWS SDK (example shown in Python-style pseudo-code).

Notes:
- In real projects, create topics/subscriptions with **IaC** (Terraform/CloudFormation) when possible.
- You’ll need IAM permissions (and real AWS account/region) for any real call.

```python
# PSEUDO-CODE (do not run)

import boto3

region = "us-east-1"
sns = boto3.client("sns", region_name=region)

# 1) Create (or get) a topic
topic = sns.create_topic(Name="orders-events")
topic_arn = topic["TopicArn"]

# 2) Subscribe a consumer (example: an SQS queue)
# (You can also subscribe Lambda, HTTP/S, email, SMS, etc.)
queue_arn = "arn:aws:sqs:us-east-1:123456789012:orders-processor"
sns.subscribe(
    TopicArn=topic_arn,
    Protocol="sqs",
    Endpoint=queue_arn,
)

# 3) Publish a message
sns.publish(
    TopicArn=topic_arn,
    Subject="order_created",
    Message='{"event":"order_created","order_id":"o_123"}',
)

# 4) (Optional) Use message attributes for filtering on subscribers
sns.publish(
    TopicArn=topic_arn,
    Message="training_completed",
    MessageAttributes={
        "event_type": {"DataType": "String", "StringValue": "ml"}
    },
)
```


## Quick pitfalls

- Delivery is typically **at-least-once**; subscribers should be **idempotent**.
- Ordering is not guaranteed unless you use **FIFO topics** (and compatible subscribers).
- Messages have size limits (commonly **256 KB**); store large payloads in S3 and publish a pointer.
